conda config --add channels conda-forge
conda create -n rag_chatbot2 python pandas nltk numpy jq "langchain<1.0" langchain-community chromadb langchain-chroma langchain-ollama langchain-openai streamlit
conda activate rag_chatbot2
pip install grobid_tei_xml rank_bm25

In [1]:
from langchain_community.document_loaders import JSONLoader, CSVLoader
import json
from langchain_text_splitters import RecursiveCharacterTextSplitter

import pandas as pd
from langchain_community.retrievers import BM25Retriever
from langchain.chains import RetrievalQA
from langchain_ollama import OllamaLLM

import os
from langchain.docstore.document import Document
import streamlit as st
import grobid_tei_xml
import re

In [2]:
from langchain_community.document_loaders import JSONLoader, CSVLoader
import json
from pprint import pprint
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_ollama.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
import pandas as pd
import os
from langchain_core.documents import Document

import grobid_tei_xml

In [3]:
# For BM25 retrievers
from langchain_ollama import OllamaLLM
from pprint import pprint
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import BM25Retriever
from langchain.chains import RetrievalQA


In [48]:
!pip install jq
!pip install rank_bm25

In [49]:
guidelines_data_sample = pd.read_csv("../results/relevant_clin_guid_data_Oct28.csv")
guidelines_data_sample.head()

,doc_id,Score,guid_text,domain_terms,OPID,word_count,source,title,url
0,1146,2.461554,Good management of wounds is essential. This i...,"['dementia', 'older', 'falls', 'elderly', 'del...",0,45525,icrc,NaN,NaN
1,3033,2.323472,Adult- and late-onset male hypogonadism: the c...,"['dementia', 'older', 'falls', 'elderly', 'agi...",1,11101,pubmed,Adult- and late-onset male hypogonadism: the c...,https://link.springer.com/content/pdf/10.1007/...
2,1441,2.285585,Rehabilitation after traumatic injury\n\nThis ...,"['dementia', 'older', 'falls', 'elderly', 'del...",3,30638,nice,Rehabilitation after traumatic injury,https://www.nice.org.uk/guidance/ng211
3,1280,2.217596,Hip fracture: management\n\nThis guideline cov...,"['dementia', 'older', 'falls', 'delirium', 'os...",7,4579,nice,Hip fracture: management,https://www.nice.org.uk/guidance/cg124
4,984,2.075389,Low-density lipoprotein LTBI Latent tuberculos...,"['dementia', 'older', 'falls', 'aging', 'osteo...",14,16952,cma,NaN,NaN


In [50]:
# Remove large documents
guidelines_data_sample1 = guidelines_data_sample[guidelines_data_sample['word_count'] < 178499]

In [51]:
#guidelines_data_sample1.to_json("../results/relevant_clin_guid_data_Oct28.jsonl", orient="records", lines=True)

In [52]:
guidelines_data_sample1.shape

(91, 9)

In [53]:
# Please read https://python.langchain.com/docs/how_to/document_loader_json/
# Define the metadata extraction function.
def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["source"] = record.get("source")
    metadata["title"] = record.get("title")
    metadata["url"] = record.get("url")
    metadata["OPID"] = record.get("OPID")
    metadata["doc_id"] = record.get("doc_id")
    metadata["word_count"] = record.get("word_count")

    return metadata

def load_jsonl_data(data_path):
    loader  = JSONLoader(
        file_path= data_path,
        jq_schema= ".",
        content_key="guid_text",
        json_lines=True,
        metadata_func=metadata_func
    )

    guidelines_data = loader.load()

    return guidelines_data

In [54]:
guidelines_doc_data = load_jsonl_data(data_path="../results/relevant_clin_guid_data_Oct28.jsonl")

In [55]:
print(len(guidelines_doc_data))

91


In [56]:
print(len(guidelines_doc_data))

91


### Extract guideline data from adult care guidelines

In [57]:
'''
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import GrobidParser


guidelines_directory = "../data/adult-care-guidelines/"

loader = GenericLoader.from_filesystem(
    "../Papers/",
    glob="*",
    suffixes=[".pdf"],
    parser=GrobidParser(segment_sentences=False),
)

docs = loader.load()
'''

'\nfrom langchain_community.document_loaders.generic import GenericLoader\nfrom langchain_community.document_loaders.parsers import GrobidParser\n\n\nguidelines_directory = "../data/adult-care-guidelines/"\n\nloader = GenericLoader.from_filesystem(\n    "../Papers/",\n    glob="*",\n    suffixes=[".pdf"],\n    parser=GrobidParser(segment_sentences=False),\n)\n\ndocs = loader.load()\n'

In [58]:
geriatric_care_dir = "../research-papers/Keyi/results/"

### Testing rough code

In [59]:
filepath = "../research-papers/Keyi/results/surgery-and-opioids-2021_4.grobid.tei.xml"
with open(filepath, "r", encoding="utf-8") as file:
    doc1 = grobid_tei_xml.parse_document_xml(file.read())
    print(doc1)


GrobidDocument(grobid_version='0.8.1', grobid_timestamp='2025-11-07T18:23+0000', header=GrobidBiblio(authors=[], index=None, id=None, unstructured=None, date=None, title='', book_title=None, series_title=None, editors=None, journal=None, journal_abbrev=None, publisher=None, institution=None, issn=None, eissn=None, volume=None, issue=None, pages=None, first_page=None, last_page=None, note=None, doi=None, pmid=None, pmcid=None, arxiv_id=None, pii=None, ark=None, istex_id=None, url=None), pdf_md5='35A27245CA342D4C03E5DAB4EEDA3475', language_code='en', citations=[GrobidBiblio(authors=[GrobidAuthor(full_name='M A Brandenburg', given_name='M', middle_name='A', surname='Brandenburg', email=None, orcid=None, affiliation=None)], index=0, id='b0', unstructured=None, date='2019-09-06', title='Prescription Opioids Are Associated With Population Mortality in US Deep South Middle-Age Non-hispanic Whites: An Ecological Time Series Study', book_title=None, series_title=None, editors=None, journal=None

In [60]:
text_parts1 = []
if doc1.abstract is not None:
    text_parts1.append(doc1.abstract)
if doc1.body:
    text_parts1.append(doc1.body) # This might include all text within the body

whole_text1= "\n\n".join(text_parts1)

In [61]:
print(text_parts1[0:5])

['Introduction There is a duty on all practitioners and professional bodies to be responsive to public and governmental concern and to be proactive in minimising public health risks relating to substances of misuse like opioids. The year-on-year reduction in general mortality has been reversed in the USA with opioid misuse being a major component of the increase in mortality  [1] . One study found that the percentage of all deaths attributable to opioids increased by 292% (from 0.4% to 1.5%) between 2001 and 2016  [2] . Opioids are used widely intraoperatively and, used safely and appropriately, still play an important role in postoperative analgesia. Nonetheless, the post-surgical use of opioids is thought to be an important source of problems. Although we cannot quantify this problem in the UK, concern expressed by individuals and UK publications indicate that the perioperative period is an important source of new persistent opioid use and this cannot be ignored  [3] . We have a duty

In [62]:
print(whole_text)

### Parse files to Document format

In [63]:
OPID_counter = 50000

for filename in os.listdir(geriatric_care_dir):
    if filename.endswith(".xml"):
        file_path = os.path.join(geriatric_care_dir, filename)
        with open(file_path, "r", encoding="utf-8") as file:
            doc_extract = grobid_tei_xml.parse_document_xml(file.read())
            text_parts = []
            try:
                if doc_extract.abstract:
                    text_parts.append(doc_extract.abstract)
            except AttributeError:
                print("Astract missing")
            try:
                if doc_extract.body:
                    text_parts.append(doc_extract.body) # This might include all text within the body
            except AttributeError:
                print("Body missing", filename)               

            whole_text= "\n\n".join(text_parts)

            doc = Document(page_content=whole_text, metadata={"source": filename, "OPID": OPID_counter})
            OPID_counter = OPID_counter + 1
            guidelines_doc_data.append(doc)
    
    if filename.endswith(".txt"):
        file_path = os.path.join(geriatric_care_dir, filename)
        with open(file_path, "r", encoding="utf-8") as file:
            text = file.read()
            doc = Document(page_content=text, metadata={"source": filename, "OPID": OPID_counter})
            OPID_counter = OPID_counter + 1
            guidelines_doc_data.append(doc)

## Chunk creation

In [64]:
# https://python.langchain.com/api_reference/text_splitters/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html

def chunk_creation(guidelines_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size= 1000,
        chunk_overlap=50,
        length_function=len,
        is_separator_regex=False
    )

    # guidelines_data is Sequence(Documents), we used transform_documents. If it was "str", we would use "create_documents"
    chunks = text_splitter.transform_documents(guidelines_data)
    print(f"Split {len(guidelines_data)} documents into {len(chunks)} chunks.")

    return chunks

sample_chunks_data = chunk_creation(guidelines_doc_data)

Split 107 documents into 19088 chunks.


In [65]:
type(sample_chunks_data[0])

langchain_core.documents.base.Document

In [67]:
# Create Retriever
retriever = BM25Retriever.from_documents(sample_chunks_data, k=30) # k - number of top documents to retrieve when querying

### With Llama-3.1 8B Model

In [125]:
# Create RAG Chain
rag_chain = RetrievalQA.from_chain_type(llm = OllamaLLM(model= "llama3.1:8b"), retriever= retriever, return_source_documents=True)

In [126]:
pprint(rag_chain.invoke(input="Please answer the following question:  What components should be included in a comprehensive geriatric assessment (CGA), and how can it improve patient outcomes?"))

{'query': 'Please answer the following question:  What components should be '
          'included in a comprehensive geriatric assessment (CGA), and how can '
          'it improve patient outcomes?',
 'result': 'The comprehensive geriatric assessment (CGA) includes domains of '
           'functional status, cognitive function, comorbidities & geriatric '
           'syndromes, polypharmacy, psychological status, social support, and '
           'nutritional status. This assessment is suggested in the practice '
           'guidelines developed by the National Comprehensive Cancer Network '
           'and ASCO. The CGA is able to predict adverse outcomes (including '
           'chemotherapy toxicity and mortality), which can help inform shared '
           'decision making.\n'
           '\n'
           'Source: "The comprehensive geriatric assessment (CGA) include '
           'domains of functional status, cognitive function, comorbidities & '
           'geriatric syndromes, poly

In [127]:
pprint(rag_chain.invoke(input="Please answer the following question: What clinical guidelines exist for managing polypharmacy in elderly patients, and how can healthcare providers minimize the risks associated with multiple medications?"))

{'query': 'Please answer the following question: What clinical guidelines '
          'exist for managing polypharmacy in elderly patients, and how can '
          'healthcare providers minimize the risks associated with multiple '
          'medications?',
 'result': "Based on the context provided, I don't know the answer to the "
           'specific question about what clinical guidelines exist for '
           'managing polypharmacy in elderly patients, and how healthcare '
           'providers can minimize the risks associated with multiple '
           'medications. The context only mentions general information about '
           'catheterization, infection prevention, and evidence-based '
           'guidelines, but it does not specifically address the management of '
           'polypharmacy in elderly patients.',
 'source_documents': [Document(metadata={'source': 'cdc', 'seq_num': 10, 'title': None, 'url': None, 'OPID': 27, 'doc_id': 269, 'word_count': 8568}, page_content='To

In [128]:
pprint(rag_chain.invoke(input="Please answer the following question: What evidence-based strategies are recommended in clinical guidelines to prevent falls in geriatric patients, and how can these be implemented in various care settings?"))

{'query': 'Please answer the following question: What evidence-based '
          'strategies are recommended in clinical guidelines to prevent falls '
          'in geriatric patients, and how can these be implemented in various '
          'care settings?',
 'result': 'The literature recommends several evidence-based strategies to '
           'prevent falls in geriatric patients. These include:\n'
           '\n'
           '1. Conducting comprehensive assessments of older adults at risk '
           'for falls using standardized tools such as the Timed Up and Go '
           '(TUG) test or the Berg Balance Scale.\n'
           '2. Identifying modifiable risk factors, including medication use '
           '(e.g., sedatives, antidepressants), vitamin D deficiency, and '
           'chronic conditions (e.g., diabetes, heart failure).\n'
           '3. Implementing multifaceted interventions that address multiple '
           'risk factors, such as exercise programs (e.g., balance train

In [129]:
pprint(rag_chain.invoke(input="Please answer the following question: What are the recommended clinical guidelines for assessing and managing chronic pain in older adults, considering their unique physiological and psychological needs?"))

{'query': 'Please answer the following question: What are the recommended '
          'clinical guidelines for assessing and managing chronic pain in '
          'older adults, considering their unique physiological and '
          'psychological needs?',
 'result': 'The recommended clinical guidelines for assessing and managing '
           'chronic pain in older adults, considering their unique '
           'physiological and psychological needs, are as follows:\n'
           '\n'
           '* Modifications and adjustments may be necessary in both approach '
           'and dosages when managing pain in older adults, especially those '
           'who are frail.\n'
           '* Age greater than 70 and/or frailty require awareness of '
           'additional factors when assessing and managing pain.\n'
           '* Frailty is a medical syndrome with multiple causes and '
           'contributors, characterized by diminished strength and endurance '
           'and reduced physiolog

### With Gemma 3N-E4B model

In [68]:
# Create RAG Chain
meditron_rag_chain = RetrievalQA.from_chain_type(llm = OllamaLLM(model= "gemma3n:e4b"), retriever= retriever, return_source_documents=True)

In [69]:
pprint(meditron_rag_chain.invoke(input="Please answer the following question:  What components should be included in a comprehensive geriatric assessment (CGA), and how can it improve patient outcomes?"))

{'query': 'Please answer the following question:  What components should be '
          'included in a comprehensive geriatric assessment (CGA), and how can '
          'it improve patient outcomes?',
 'result': '## Components of a Comprehensive Geriatric Assessment (CGA) and '
           'its Impact on Patient Outcomes\n'
           '\n'
           'A Comprehensive Geriatric Assessment (CGA) is a thorough, '
           "multidisciplinary evaluation of an older adult's physical, "
           'psychological, and social functioning. It goes beyond simply '
           'identifying medical conditions and aims to understand the whole '
           'person and their needs. \n'
           '\n'
           '**Key Components of a CGA:**\n'
           '\n'
           'A well-rounded CGA typically includes the following elements:\n'
           '\n'
           '* **Medical History Review:** Detailed history of past and present '
           'medical conditions, medications, allergies, and previous '


In [132]:
pprint(meditron_rag_chain.invoke(input="Please answer the following question: What clinical guidelines exist for managing polypharmacy in elderly patients, and how can healthcare providers minimize the risks associated with multiple medications?"))

{'query': 'Please answer the following question: What clinical guidelines '
          'exist for managing polypharmacy in elderly patients, and how can '
          'healthcare providers minimize the risks associated with multiple '
          'medications?',
 'result': 'The provided text discusses guidelines for preventing '
           'healthcare-associated infections, particularly urinary tract '
           'infections (UTIs) related to urinary catheterization. It '
           'emphasizes the importance of education for patients, their '
           'families, and healthcare workers regarding infection prevention, '
           'including recognizing UTI symptoms and seeking expert help. \n'
           '\n'
           'However, the text **does not** contain information about clinical '
           'guidelines for managing polypharmacy in elderly patients or how '
           'healthcare providers can minimize the risks associated with '
           'multiple medications. \n'
           '\n

In [133]:
pprint(meditron_rag_chain.invoke(input="Please answer the following question: What evidence-based strategies are recommended in clinical guidelines to prevent falls in geriatric patients, and how can these be implemented in various care settings?"))

{'query': 'Please answer the following question: What evidence-based '
          'strategies are recommended in clinical guidelines to prevent falls '
          'in geriatric patients, and how can these be implemented in various '
          'care settings?',
 'result': 'According to the provided text, the following evidence-based '
           'strategies are recommended in clinical guidelines to prevent falls '
           'in geriatric patients:\n'
           '\n'
           '*   **Falls risk screening tools:** Utilize tools available on the '
           'British Columbia Injury Research and Prevention Unit (BCIRPU) '
           'website to identify clients with risk factors.\n'
           '*   **Comprehensive assessment:**  Implement further comprehensive '
           'assessment by the interprofessional team for clients identified as '
           'having risk factors.\n'
           '*   **Targeted falls prevention strategies:** Develop and '
           'implement individualized plans

In [134]:
pprint(meditron_rag_chain.invoke(input="Please answer the following question: What are the recommended clinical guidelines for assessing and managing chronic pain in older adults, considering their unique physiological and psychological needs?"))

{'query': 'Please answer the following question: What are the recommended '
          'clinical guidelines for assessing and managing chronic pain in '
          'older adults, considering their unique physiological and '
          'psychological needs?',
 'result': 'The committee recommends reasonable clinical judgement, clear '
           'documentation, and frequent reassessment for managing chronic pain '
           'in older adults. They emphasize that managing pain often requires '
           'modifications and adjustments in both approach and dosages, '
           'considering age greater than 70 and/or frailty. Pharmacological '
           "management is often a part of overall care, and it's imperative to "
           'get acute pain under control. However, the evidence suggests that '
           'the benefits of opioids for pain relief and physical functioning '
           'are small. The committee highlights that no guideline can account '
           'for the unique features

## Changing from BM25 to Qwen3 embeddings and Chroma Vector Database

In [ ]:
# ollama pull qwen3-embedding:8b.  --- https://ollama.com/library/qwen3-embedding:8b

embeddings = OllamaEmbeddings(model="qwen3-embedding:8b")
vector_store = Chroma(
    collection_name="clinical_guidelines_adult_care",
    embedding_function=embeddings,
    persist_directory= "../chroma_dbs/"
)

# You should usually provide IDs for the documents you add to the vector store, so that instead of adding the same document multiple times, you can update the existing document.
# Uncomment if running for the first time, i.e., the Chroma vectorDB is not yet created. OR if you want to add some more documents
vector_store.add_documents(documents=sample_chunks_data)

# Adding a Streamlit Interface

In [ ]:
import streamlit as st

st.set_page_config(page_title="RAG Chatbot for Geriatric Care")

st.title("RAG Chatbot for Geriatric Care")

In [ ]:
# Input text box for user query
user_query = st.text_input("Ask a question:")


# Button to submit the query
if st.button("Submit"):
    if user_query:
        # Get the answer from the RetrievalQA instance
        answer = meditron_rag_chain.invoke(input= user_query)
        
        # Display the answer
        st.write("Answer:")
        st.write(answer['result'])
    else:
        st.write("Please enter a question.")

# Old Code

In [17]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}
 - -
Answer the question based on the above context: {question}
"""

In [18]:
def query_rag(query, vector_db, top_k=3):
    # Retrieve the relevant context from vector_db based on query
    results= vector_db.similarity_search(query, k=top_k) # additional parameter: filter (dict | None) – Dictionary of argument(s) to filter on metadata

    # Combine context from matching documents
    context_text = "\n\n - -\n\n".join([doc.page_content for doc in results])
    #print("Context: ", context_text)

    prompt= ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    #prompt= prompt_template.format(context= context_text, question= query)

    #llm = ChatOpenAI(model="gpt-4.1-mini")
    llm = ChatOllama(model="gemma3:27b")

    chain = prompt | llm | StrOutputParser()

    return chain.invoke({
        "context": context_text,
        "question": query
    }), context_text

In [19]:
query = "In what ways does PET or PET/CT contribute to the assessment of treatment response and detection of recurrence in patients with head and neck cancer, especially when conventional imaging results are equivocal?"


How does PET or PET/CT imaging improve the accuracy of staging in patients with head and neck squamous cell carcinoma compared to conventional imaging methods like CT or MRI?

What is the clinical value of PET or PET/CT in identifying the primary tumor site in patients presenting with cervical lymph node metastases but an unknown primary tumor?

In what ways does PET or PET/CT contribute to the assessment of treatment response and detection of recurrence in patients with head and neck cancer, especially when conventional imaging results are equivocal?

What role does PET imaging play in the management of patients with nasopharyngeal carcinoma, particularly regarding staging and detection of distant metastatic disease?

How does PET-directed management after radiation or chemoradiation therapy influence decisions about neck dissection in patients with residual lymphadenopathy on conventional imaging?

In [21]:
response, context_text= query_rag(query= query, vector_db= vector_store)
print(response)

KeyboardInterrupt: 

In [44]:
response, context_text= query_rag(query= "What is the clinical value of PET or PET/CT in identifying the primary tumor site in patients presenting with cervical lymph node metastases but an unknown primary tumor?", vector_db= vector_store)
print(response)

Based on the provided context, the clinical value of PET or PET/CT in patients presenting with cervical lymph node metastases but an unknown primary tumor is as follows:

- PET can detect the primary tumor site in approximately 30% of such patients, including tumors that are missed by conventional imaging methods.
- PET is recommended after conventional imaging and in addition to, or prior to, diagnostic panendoscopy when the primary site is unknown.
- PET is better than conventional imaging in detecting the site of the primary tumor, as supported by primary studies.
- PET can also detect some synchronous primary tumors that might be missed by other imaging modalities.

In summary, PET or PET/CT improves the identification of unknown primary tumors in patients with cervical lymph node metastases, thus aiding in diagnosis and potentially influencing subsequent management.


In [45]:
response, context_text= query_rag(query= "How does PET-directed management after radiation or chemoradiation therapy influence decisions about neck dissection in patients with residual lymphadenopathy on conventional imaging?", vector_db= vector_store)
print(response)

Based on the provided context, PET-directed management after radiation or chemoradiation therapy influences decisions about neck dissection in patients with residual lymphadenopathy on conventional imaging by appropriately sparing neck dissections in those patients who have PET-negative residual abnormalities on CT. In other words, if conventional imaging shows residual lymphadenopathy but the PET scan is negative, neck dissection can often be avoided, reducing unnecessary surgery. This approach is supported by moderate evidence.
